# Homework 3 - Interactive Viz

In [ ]:
import folium
import requests as rq
from bs4 import BeautifulSoup as bfs
import numpy as np
import pandas as pd
import collections
import os
%matplotlib inline
import matplotlib.pyplot as plt
from dateutil import relativedelta
import scipy.stats as stats
import math

## Test Map

In [ ]:
state_geo = r'ch-cantons.topojson.json'

#Let Folium determine the scale
map = folium.Map(location=[46.8, 7.1], zoom_start=8)
#map.choropleth(geo_path=state_geo, fill_color='YlGn', fill_opacity=0.7, line_opacity=0.2)
#map.save('us_states.html')
map

## SNSF data

### Grant CSV file

In [ ]:
snsf_data = pd.read_csv('Data/P3_GrantExport.csv', sep=';', parse_dates=['Start Date', 'End Date'])
snsf_data.head()

In [ ]:
snsf_data.dtypes

In [ ]:
snsf_data.columns = snsf_data.columns.str.replace(snsf_data.columns[0], 'Project Number')
snsf_data.head()

In [ ]:
snsf_data['Project Number'].is_unique

In [ ]:
snsf_data.set_index('Project Number').head()

In [ ]:
snsf_data['University'].value_counts(dropna=False).head()
#snsf_data['University'].value_uniques()

## Test with differences slices of years.

From the doc : "To a lesser extent, there may also be a problem with data quality,
for example data on fellowships for prospective researchers is incomplete for the 1990’s. Moreover,
data on SFGBM for advanced researchers is incomplete before 2005."

> J'essai de voir si il y a des differences entre les années car les données devraient étre plus pertinentes à partir de 2000's

In [ ]:
limit_date = np.datetime64('2005');
snsf_data_2005_2018 = snsf_data[snsf_data['Start Date'] >= limit_date ]
snsf_data_2005_2018.head()
snsf_data_2005_2018['University'].value_counts(dropna=False).head()

In [ ]:
limit_date_sup = np.datetime64('2005');
limit_date_inf = np.datetime64('2000');

snsf_data_2000_2005 = snsf_data[(snsf_data['Start Date'] < limit_date_sup) & (snsf_data['Start Date'] >= limit_date_inf) ]
snsf_data_2000_2005.head()
snsf_data_2000_2005['University'].value_counts(dropna=False).head()

In [ ]:
limit_date_sup = np.datetime64('2000');

snsf_data_1975_2000 = snsf_data[(snsf_data['Start Date'] < limit_date_sup)]
snsf_data_1975_2000.head()
snsf_data_1975_2000['University'].value_counts(dropna=False).head()

> --> ça change pas grand chose...

### Test blanck attribute.

University field : "This is the institution where the project will largely be carried out according to the application. Pick list. This field is only filled if the research is carried out at a Swiss institution, otherwise the field remains blank. In the case of mobility fellowships, it is generally left empty."

> Je cheche quelles sont les attirbuts qui contiennent la "mobility fellowship" car c'est peut etre des universités suisse.

In [ ]:
snsf_data['Funding Instrument Hierarchy'].value_counts(dropna=False)

In [ ]:
snsf_data[snsf_data['Funding Instrument'].str.contains("fellowships")]['Funding Instrument'].value_counts(dropna=False)

In [ ]:
snsf_data[snsf_data['Funding Instrument'].str.contains("Mobility")]['Funding Instrument'].value_counts(dropna=False)

In [ ]:
index_link_mobility = snsf_data[snsf_data['Funding Instrument'].str.contains("Mobility")]['Funding Instrument'].value_counts(dropna=False).index

In [ ]:
index_array = []
for mobility_attribute in index_link_mobility:
    index_array.append(mobility_attribute)
index_array

In [ ]:
data_related_to_mobility_fellowship = snsf_data[snsf_data['Funding Instrument'].isin(index_array)].copy()
data_related_to_mobility_fellowship['University'].value_counts(dropna=False)

In [ ]:
data_related_to_mobility_fellowship['Institution'].value_counts(dropna=False)

> --> A chercher plus en profondeur il doit avoir surement quelque chose avec la bouse mobility. Pourquoi il y a des universités suisse ??


In [ ]:
# We keep only the university not null, because those university are swiss.
snsf_data_swiss_Data = snsf_data[snsf_data['University'].notnull()].copy()
snsf_data_swiss_Data.head()

### Other CSV files

#### Publication

In [ ]:
snsf_publication_data = pd.read_csv('Data/P3_PublicationExport.csv', sep=';')
snsf_publication_data.head()

#### Person

In [ ]:
snsf_person_data = pd.read_csv('Data/P3_PersonExport.csv', sep=';')
snsf_person_data.head()

> On a aussi un champ représentant un lieu "Institue Place" mais j'ai peur que l'on s'éloigne trop loin en effectuant un merge (un auteur peut contribuer dans différents projets à différents endroits)

## GeoNames

> Test de la lib GeoPy pour faire les appels à GeoNames, mais n'est pas terrible du tout !

> Il faut asbolument installer "geopy" dans anaconda pour s'en servir (il est déjà présent dans le channel "default")

In [ ]:
from geopy.geocoders import GeoNames

# Limit of 2000 request per hour
geolocator = GeoNames(username='ada_epfl_insa')
result = geolocator.geocode('Université de Genève - GE') # Pas de résultat ... Il faut un code postal
print(result)

> C'est mieux d'utiliser Request, on n'est plus limité pour les options !

In [ ]:
payload = {
    'country': 'CH',
    'postalcode': 'GE',
    'placename': 'Université de Genève',
    'maxRows': 10,
    'username': 'ada_epfl_insa'
}
request_test_page = rq.get('http://api.geonames.org/postalCodeSearch', payload)
test_page = bfs(request_test_page.text, 'xml')
test_page

> Bon c'est pas terrible du tout GeoNames ............